In [1]:
import numpy as np
from tqdm import tqdm
import os
import pandas as pd
from torch.utils.data import Dataset
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

O_size=720
class DFTDataset(Dataset):
    def __init__(self, 
                 data, 
                 window_size=96,
                 O_size=720,
                 flip=True,
                 norm=True):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.mean = data.mean()
        self.std = data.std()
        if norm:
            self.data = self.transform(data)
        else:
            self.data = Data
        self.window_size = window_size

        self.O_size = O_size
        self.flip = flip
        self.dataset_len = len(self.data) - window_size - O_size + 1
        self.slicesX_ = np.zeros((self.dataset_len, window_size))
        self.slicesT_ = np.zeros((self.dataset_len, O_size))

        # 构建输入和目标序列
        for i in range(self.dataset_len):
            self.slicesX_[i] = self.data[i:i + window_size]
            self.slicesT_[i] = self.data[i + window_size:i + window_size + O_size]
        
        self.slicesX = self.slicesX_
        self.slicesT = self.slicesT_
        self.input_DFTtri = self.slices2DFTtri(self.slicesX)
        self.slicesT = self.slicesT
        #self.input_DFTtri = ei.rearrange(self.slices2DFTtri(self.slicesX),'b h w c -> b c h w')
        

    def __len__(self):
        return self.dataset_len

    def __getitem__(self, idx):
        input_DFTtri = self.input_DFTtri[idx]
        target_seq = self.slicesT[idx]
        
        return input_DFTtri, target_seq
    def slices2DFTtri(self, 
                      slicesX, 
                      without_f0=False):
        timeSteps = slicesX.shape[0]
        windowSize = slicesX.shape[1]
        max_freq = int((windowSize + 3) / 2)  # int()函数只保留整数部分
        # complex128就是64+64的复数
        # timestep是矩阵的数量，后面两个参数决定了每个矩阵的维度，生成1247个12*12的matrix
        DFTtri = np.zeros([timeSteps, windowSize, windowSize], dtype = np.complex64) #降低精度防止爆内存
        for i in tqdm(range(timeSteps),desc="Loading Data",ncols=100):
            for j in range(windowSize):
                fft = np.fft.fft(slicesX[i, -(1+j):])
                DFTtri[i, :(j+1), j] = fft[::-1]
                if self.flip:
                    DFTtri[i, j, :(j+1)] = fft[::-1] # Flip padding
        if without_f0:
            DFTtri = DFTtri[:,1:,1:]
        DFTtriDescartes = np.stack([DFTtri.real, DFTtri.imag], axis=-3)
        return DFTtriDescartes
    def transform(self, data):
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean

class MAN(nn.Module):
    def __init__(self, num_classes=O_size):
        super(MAN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size=3, stride=1, padding=1),  
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 192, kernel_size=3, stride=1, padding=1),  
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 256, kernel_size=3, stride=1, padding=1),  
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
def main():
    # current_dir = os.path.dirname(os.path.realpath(__file__))
    # os.chdir(current_dir)

    etdata = pd.read_csv('./ETTm2.csv')
    ot = etdata['OT'].values


    dft_dataset = DFTDataset(ot)

    dataset_size = len(dft_dataset)
    train_size = int(0.7 * dataset_size)
    val_size = int(0.1 * dataset_size)
    test_size = dataset_size - train_size - val_size
    dataloader = torch.utils.data.DataLoader(dft_dataset, 
                                             batch_size=256, 
                                             shuffle=False,
                                             pin_memory=True)

    model = MAN()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 50

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    try:
        checkpoint = torch.load('./save_models/model_840.pth')
        model.load_state_dict(checkpoint['model'])
    except:
        pass
    for epoch in range(num_epochs):
        model.train()
        running_loss, batch_loss, vali_loss, test_loss = 0,0,0,0
        count=0
        for inputs, targets in tqdm(dataloader, 
                                    desc=f'Epoch {epoch + 1}/{num_epochs}',
                                    ncols=100, 
                                    leave=False):
            inputs, targets = inputs.float().to(device), targets.float().to(device)
            if count< train_size:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                invtrans_outputs = dft_dataset.inverse_transform(outputs)
                invtrans_targets = dft_dataset.inverse_transform(targets)
                batch_loss = criterion(invtrans_outputs, invtrans_targets)
                norm_train_loss = loss.item()
                running_loss += batch_loss.item() * inputs.size(0)
            elif count>= train_size and count< train_size+val_size:
                model.eval()
                outputs = model(inputs)
                norm_vali_loss = criterion(outputs,targets).item()
                invtrans_outputs = dft_dataset.inverse_transform(outputs)
                invtrans_targets = dft_dataset.inverse_transform(targets)
                loss = criterion(invtrans_outputs, invtrans_targets)
                vali_loss += loss.item() * inputs.size(0)
            elif count>= train_size+val_size:
                model.eval()
                outputs = model(inputs)
                norm_test_loss = criterion(outputs,targets).item()
                invtrans_outputs = dft_dataset.inverse_transform(outputs)
                invtrans_targets = dft_dataset.inverse_transform(targets)
                loss = criterion(invtrans_outputs, invtrans_targets)
                test_loss += loss.item() * inputs.size(0)
            count+=inputs.size(0)
        epoch_loss = running_loss / train_size
        vali_loss = vali_loss / val_size
        test_loss = test_loss / test_size
        print(f"Epoch {epoch + 1}/{num_epochs}, Train_Loss: {epoch_loss:.4f}/{norm_train_loss:.4f}, \
              Vali_Loss:{vali_loss}/{norm_vali_loss:.4f}, Test_Loss:{test_loss}/{norm_test_loss:.4f}")
        if epoch %10 ==0:
            checkpoint = {'epoch':epoch,
            
                          'model':model.state_dict(),
                          'optimizer':optimizer.state_dict()}
            torch.save(checkpoint,f"./save_models/model_{epoch}.pth")
    print("Finished Training")
    

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def main():
    etdata = pd.read_csv('./ETTm2.csv')
    ot = etdata['OT'].values

    dft_dataset = DFTDataset(ot)

    dataset_size = len(dft_dataset)
    train_size = int(0.7 * dataset_size)
    val_size = int(0.1 * dataset_size)
    test_size = dataset_size - train_size - val_size
    dataloader = DataLoader(dft_dataset, 
                             batch_size=256, 
                             shuffle=False,
                             pin_memory=True)

    model = MAN()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 50

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    try:
        checkpoint = torch.load('./save_models/model_840.pth')
        model.load_state_dict(checkpoint['model'])
    except:
        pass
    
    # Initialize TensorBoard writer
    writer = SummaryWriter('logs')

    for epoch in range(num_epochs):
        model.train()
        running_loss, batch_loss, vali_loss, test_loss = 0,0,0,0
        count=0
        for inputs, targets in tqdm(dataloader, 
                                    desc=f'Epoch {epoch + 1}/{num_epochs}',
                                    ncols=100, 
                                    leave=False):
            inputs, targets = inputs.float().to(device), targets.float().to(device)
            if count< train_size:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                invtrans_outputs = dft_dataset.inverse_transform(outputs)
                invtrans_targets = dft_dataset.inverse_transform(targets)
                batch_loss = criterion(invtrans_outputs, invtrans_targets)
                norm_train_loss = loss.item()
                running_loss += batch_loss.item() * inputs.size(0)
            elif count>= train_size and count< train_size+val_size:
                model.eval()
                outputs = model(inputs)
                norm_vali_loss = criterion(outputs,targets).item()
                invtrans_outputs = dft_dataset.inverse_transform(outputs)
                invtrans_targets = dft_dataset.inverse_transform(targets)
                loss = criterion(invtrans_outputs, invtrans_targets)
                vali_loss += loss.item() * inputs.size(0)
                
                # Plot the predicted and real values of the last item of the last batch of the validation set
                if count + inputs.size(0) == val_size:
                    last_output = invtrans_outputs[-1].detach().cpu().numpy()
                    last_target = invtrans_targets[-1].detach().cpu().numpy()
                    print(last_output)
                    fig, ax = plt.subplots()
                    ax.plot(last_output, label='Predicted')
                    ax.plot(last_target, label='Real')
                    ax.legend()
                    writer.add_figure('Validation/Predicted_vs_Real', fig, global_step=epoch)
                    
            elif count>= train_size+val_size:
                model.eval()
                outputs = model(inputs)
                norm_test_loss = criterion(outputs,targets).item()
                invtrans_outputs = dft_dataset.inverse_transform(outputs)
                invtrans_targets = dft_dataset.inverse_transform(targets)
                loss = criterion(invtrans_outputs, invtrans_targets)
                test_loss += loss.item() * inputs.size(0)
            count+=inputs.size(0)
        
        epoch_loss = running_loss / train_size
        vali_loss = vali_loss / val_size
        test_loss = test_loss / test_size
        
        # Log losses to TensorBoard
        writer.add_scalar('Loss/train', epoch_loss, epoch)
        writer.add_scalar('Loss/validation', vali_loss, epoch)
        writer.add_scalar('Loss/test', test_loss, epoch)
        
        print(f"Epoch {epoch + 1}/{num_epochs}, Train_Loss: {epoch_loss:.4f}/{norm_train_loss:.4f}, \
              Vali_Loss:{vali_loss}/{norm_vali_loss:.4f}, Test_Loss:{test_loss}/{norm_test_loss:.4f}")
        
        if epoch % 50 == 0:
            checkpoint = {
                'epoch': epoch,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict()
            }
            torch.save(checkpoint, f"./save_models/model_{epoch}.pth")
    
    print("Finished Training")
    writer.close()

if __name__ == '__main__':
    main()


Loading Data: 100%|█████████████████████████████████████████| 68865/68865 [00:25<00:00, 2731.61it/s]


Epoch 1/50, Train_Loss: 86.8776/0.1912,               Vali_Loss:42.52774871285489/0.7794, Test_Loss:200.6493088262632/3.0306


KeyboardInterrupt: 

In [6]:
last_output

NameError: name 'last_output' is not defined

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class DFTDataset(Dataset):
    def __init__(self, 
                 data, 
                 window_size=96,
                 O_size=720,
                 flip=True,
                 norm=True):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.mean = data.mean()
        self.std = data.std()
        if norm:
            self.data = self.transform(data)
        else:
            self.data = Data
        self.window_size = window_size

        self.O_size = O_size
        self.flip = flip
        self.dataset_len = len(self.data) - window_size - O_size + 1
        self.slicesX_ = np.zeros((self.dataset_len, window_size))
        self.slicesT_ = np.zeros((self.dataset_len, O_size))

        # 构建输入和目标序列
        for i in range(self.dataset_len):
            self.slicesX_[i] = self.data[i:i + window_size]
            self.slicesT_[i] = self.data[i + window_size:i + window_size + O_size]
        
        self.slicesX = self.slicesX_
        self.slicesT = self.slicesT_
        self.input_DFTtri = self.slices2DFTtri(self.slicesX)
        self.slicesT = self.slicesT
        #self.input_DFTtri = ei.rearrange(self.slices2DFTtri(self.slicesX),'b h w c -> b c h w')
        

    def __len__(self):
        return self.dataset_len

    def __getitem__(self, idx):
        input_DFTtri = self.input_DFTtri[idx]
        target_seq = self.slicesT[idx]
        
        return input_DFTtri, target_seq
    def slices2DFTtri(self, 
                      slicesX, 
                      without_f0=False):
        timeSteps = slicesX.shape[0]
        windowSize = slicesX.shape[1]
        max_freq = int((windowSize + 3) / 2)  # int()函数只保留整数部分
        # complex128就是64+64的复数
        # timestep是矩阵的数量，后面两个参数决定了每个矩阵的维度，生成1247个12*12的matrix
        DFTtri = np.zeros([timeSteps, windowSize, windowSize], dtype = np.complex64) #降低精度防止爆内存
        for i in tqdm(range(timeSteps),desc="Loading Data",ncols=100):
            for j in range(windowSize):
                fft = np.fft.fft(slicesX[i, -(1+j):])
                DFTtri[i, :(j+1), j] = fft[::-1]
                if self.flip:
                    DFTtri[i, j, :(j+1)] = fft[::-1] # Flip padding
        if without_f0:
            DFTtri = DFTtri[:,1:,1:]
        DFTtriDescartes = np.stack([DFTtri.real, DFTtri.imag], axis=-3)
        return DFTtriDescartes
    def transform(self, data):
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean
class MAN(nn.Module):
    def __init__(self, num_classes=96):
        super(MAN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size=3, stride=1, padding=1),  
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 192, kernel_size=3, stride=1, padding=1),  
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 256, kernel_size=3, stride=1, padding=1),  
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [61]:
class DFTTRI:
    def __init__(self, pred_len, num_epochs=50, lr=1e-3): 
        self.pred_len = pred_len                                                                            
        self.model = MAN(num_classes=pred_len)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)     
        self.num_epochs = num_epochs
        self.writer = SummaryWriter('logs')
        self.epoch = 0
        self.loss = 0

    def load_data(self, ot, name='edt', batch_size=256):
        self.name = name + str(self.pred_len)
        dataset_size = ot.shape[0]
        self.train_size = int(0.7 * dataset_size)
        self.val_size = int(0.1 * dataset_size)
        self.test_size = dataset_size - self.train_size - self.val_size
        

        self.trainset = DFTDataset(ot[:self.train_size],O_size=self.pred_len)
        self.valiset = DFTDataset(ot[self.train_size:self.train_size+self.val_size],O_size=self.pred_len)
        self.testset = DFTDataset(ot[self.train_size+self.val_size:],O_size=self.pred_len)
        self.trainloader = torch.utils.data.DataLoader(self.trainset, 
                                                batch_size=batch_size,
                                                shuffle=False,
                                                pin_memory=True)
        self.valiloader = torch.utils.data.DataLoader(self.valiset, 
                                                batch_size=batch_size, 
                                                shuffle=False,
                                                pin_memory=True)
        self.testloader = torch.utils.data.DataLoader(self.testset, 
                                                batch_size=batch_size, 
                                                shuffle=False,
                                                pin_memory=True)   
    def mse(self, outputs, targets):
        return ((outputs - targets) ** 2).mean().item()

    def mae(self, outputs, targets):
        return (outputs - targets).abs().mean().item()
    
    def metric(self, outputs, targets):
        return self.mse(outputs, targets), self.mae(outputs, targets)

    def _plot(self, outputs, targets, mode):
        final_output = outputs[-1]
        final_target = targets[-1]
        # 绘制final_output和final_target，并保存到writer的logs中
        fig, ax = plt.subplots()
        ax.plot(final_output.detach().cpu().numpy(), label='Final Output')
        ax.plot(final_target.detach().cpu().numpy(), label='Final Target')
        ax.legend()
        self.writer.add_figure(f'{self.name}/{mode}/Final_Output_vs_Target', fig, global_step=self.epoch)
    
    def train(self):
        self.model.train()
        mse_loss, mae_loss, inv_mse_loss, inv_mae_loss = 0, 0, 0, 0
        for inputs, targets in self.trainloader:
            inputs, targets = inputs.float().to(self.device), targets.float().to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.criterion(outputs, targets)
            loss.backward()
            self.optimizer.step()
            invtrans_outputs = self.trainset.inverse_transform(outputs)
            invtrans_targets = self.trainset.inverse_transform(targets)
            ori_loss = self.metric(outputs, targets)
            inv_loss = self.metric(invtrans_outputs, invtrans_targets)
            mse_loss += ori_loss[0]
            mae_loss += ori_loss[1]
            inv_mse_loss += inv_loss[0]
            inv_mae_loss += inv_loss[1]
        
        mse_loss /= len(self.trainloader)
        mae_loss /= len(self.trainloader)
        inv_mse_loss /= len(self.trainloader)
        inv_mae_loss /= len(self.trainloader)
        
        self.loss = mse_loss 
        
        self.writer.add_scalar(f'{self.name}/Train_Loss/mse', mse_loss, self.epoch)
        self.writer.add_scalar(f'{self.name}/Train_Loss/mae', mae_loss, self.epoch)
        self.writer.add_scalar(f'{self.name}/Train_Loss/mse_inv', inv_mse_loss, self.epoch)
        self.writer.add_scalar(f'{self.name}/Train_Loss/mae_inv', inv_mae_loss, self.epoch)
        self._plot(outputs, targets, 'train')
    
    def vali(self):
        self.model.eval()
        mse_loss, mae_loss, inv_mse_loss, inv_mae_loss = 0, 0, 0, 0
        with torch.no_grad():
            for inputs, targets in self.valiloader:
                inputs, targets = inputs.float().to(self.device), targets.float().to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
                invtrans_outputs = self.valiset.inverse_transform(outputs)
                invtrans_targets = self.valiset.inverse_transform(targets)
                ori_loss = self.metric(outputs, targets)
                inv_loss = self.metric(invtrans_outputs, invtrans_targets)
                mse_loss += ori_loss[0]
                mae_loss += ori_loss[1]
                inv_mse_loss += inv_loss[0]
                inv_mae_loss += inv_loss[1]

        mse_loss /= len(self.valiloader)
        mae_loss /= len(self.valiloader)
        inv_mse_loss /= len(self.valiloader)
        inv_mae_loss /= len(self.valiloader)
        self.writer.add_scalar(f'{self.name}/Vali_Loss/mse', mse_loss, self.epoch)
        self.writer.add_scalar(f'{self.name}/Vali_Loss/mae', mae_loss, self.epoch)
        self.writer.add_scalar(f'{self.name}/Vali_Loss/mse_inv', inv_mse_loss, self.epoch)
        self.writer.add_scalar(f'{self.name}/vali_Loss/mae_inv', inv_mae_loss, self.epoch)
        self._plot(outputs, targets, 'vali')

    def test(self):
        self.model.eval()
        mse_loss, mae_loss, inv_mse_loss, inv_mae_loss = 0, 0, 0, 0
        with torch.no_grad():
            for inputs, targets in self.testloader:
                inputs, targets = inputs.float().to(self.device), targets.float().to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
                invtrans_outputs = self.testset.inverse_transform(outputs)
                invtrans_targets = self.testset.inverse_transform(targets)
                ori_loss = self.metric(outputs, targets)
                inv_loss = self.metric(invtrans_outputs, invtrans_targets)
                mse_loss += ori_loss[0]
                mae_loss += ori_loss[1]
                inv_mse_loss += inv_loss[0]
                inv_mae_loss += inv_loss[1]

        mse_loss /= len(self.testloader)
        mae_loss /= len(self.testloader)
        inv_mse_loss /= len(self.testloader)
        inv_mae_loss /= len(self.testloader)
        self.writer.add_scalar(f'{self.name}/Test_Loss/mse', mse_loss, self.epoch)
        self.writer.add_scalar(f'{self.name}/Test_Loss/mae', mae_loss, self.epoch)
        self.writer.add_scalar(f'{self.name}/Test_Loss/mse_inv', inv_mse_loss, self.epoch)
        self.writer.add_scalar(f'{self.name}/Test_Loss/mae_inv', inv_mae_loss, self.epoch)
        self._plot(outputs, targets, 'test')

    def run(self):
        with tqdm(total=self.num_epochs, desc='Training', ncols=100, leave=False) as pbar:
            for self.epoch in range(self.num_epochs):
                self.train()
                self.vali()
                self.test()
                pbar.set_description(f'Training (Loss: {self.loss:.4f})')
                pbar.update(1)

                if self.epoch % 50 == 0 and self.epoch:
                    # 保存模型
                    torch.save(self.model.state_dict(), f"./save_models/model_{self.epoch}.pth")

    


In [62]:
etdata = pd.read_csv('./ETTm2.csv')
ot = etdata['OT'].values

In [63]:
dfttri = DFTTRI(num_epochs=1000 , pred_len=720, lr=1e-2)

In [64]:
dfttri.load_data(ot, name='edt', batch_size=1024)

Loading Data: 100%|█████████████████████████████████████████| 13121/13121 [00:04<00:00, 2707.09it/s]


In [65]:
dfttri.run()

Training (Loss: 1.0072):   1%|▎                                  | 8/1000 [01:54<3:56:52, 14.33s/it]